In [31]:
# !pip install dimod
import math

from openfermionpyscf import run_pyscf
from openfermion.transforms import binary_code_transform, bravyi_kitaev_code, get_fermion_operator
# from openfermion.hamiltonians import MolecularData
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.utils import count_qubits
from pyscf import gto, scf, mcscf

from helper_functions import *
from XBK_method import *
from QCC_method import *

from openfermion.chem import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.linalg import get_ground_state, get_sparse_operator
import numpy
import scipy
import scipy.linalg

In [23]:
#create molecule
name = 'H3'
charge = 1
multiplicity = 1
basis = 'sto-6g'

bond_length = 1.1
geometry = get_molGeometry(name, bond_length)
    
molecule = MolecularData(
    geometry=geometry,
    basis=basis,
    multiplicity=multiplicity,
    charge=charge
)

In [24]:
#run RHF calculations
driver = run_pyscf(molecule, run_scf=True)
hf_energy = float(molecule.hf_energy)
hf_data = molecule._pyscf_data['scf']

# print(molecule.canonical_orbitals)
print(hf_energy)

-1.2400693045269726


In [25]:
#define active space
n_active_electrons = 2
n_active_orbitals = 3
occupied_indices, active_indices = get_active_space(molecule, n_active_electrons, n_active_orbitals)

#run CASCI calculations
casci_data = hf_data.CASCI(n_active_orbitals, n_active_electrons).run(verbose=False)
casci_energy = float(casci_data.e_tot)

print(casci_energy)

-1.2722647705358243


In [26]:
#convert to fermionic Hamiltonian
molecular_H = molecule.get_molecular_hamiltonian(occupied_indices=occupied_indices, active_indices=active_indices)
if molecular_H[()] == None:
    molecular_H[()] = 0
fermionic_H = get_fermion_operator(molecular_H)
# print(fermionic_H)
#add penalty term to ensure correct number of electrons in ground state
weight = 5
penalty_term = FermionOperator('', n_active_electrons)

for i in range(molecular_H.n_qubits):
    penalty_term += FermionOperator(str(i)+'^ '+str(i), -1)
fermionic_H += weight*penalty_term**2

print(fermionic_H)

21.443210575236364 [] +
-21.623075395384372 [0^ 0] +
5.0 [0^ 0 0^ 0] +
5.0 [0^ 0 1^ 1] +
5.0 [0^ 0 2^ 2] +
5.0 [0^ 0 3^ 3] +
5.0 [0^ 0 4^ 4] +
5.0 [0^ 0 5^ 5] +
0.2814354555027043 [0^ 0^ 0 0] +
0.07258994740907986 [0^ 0^ 2 2] +
0.07258994740907997 [0^ 0^ 4 4] +
0.2814354555027043 [0^ 1^ 1 0] +
0.07258994740907986 [0^ 1^ 3 2] +
0.07258994740907997 [0^ 1^ 5 4] +
0.07258994740907986 [0^ 2^ 0 2] +
0.27480121211888686 [0^ 2^ 2 0] +
0.0401746698698935 [0^ 2^ 2 2] +
-0.029525617108668308 [0^ 2^ 2 4] +
-0.029525617108668314 [0^ 2^ 4 2] +
-0.040174669869893515 [0^ 2^ 4 4] +
0.07258994740907986 [0^ 3^ 1 2] +
0.27480121211888686 [0^ 3^ 3 0] +
0.0401746698698935 [0^ 3^ 3 2] +
-0.029525617108668308 [0^ 3^ 3 4] +
-0.029525617108668314 [0^ 3^ 5 2] +
-0.040174669869893515 [0^ 3^ 5 4] +
0.07258994740907997 [0^ 4^ 0 4] +
-0.029525617108668314 [0^ 4^ 2 2] +
-0.040174669869893515 [0^ 4^ 2 4] +
0.27480121211888714 [0^ 4^ 4 0] +
-0.04017466986989355 [0^ 4^ 4 2] +
0.02952561710866843 [0^ 4^ 4 4] +
0.07258994

In [27]:
#convert to Pauli operator Hamiltonian
binary_code = bravyi_kitaev_code(molecular_H.n_qubits)
qubit_H = binary_code_transform(fermionic_H, binary_code)
qubit_H.compress()

#apply symmetry reductions and calculate minimum eigenvalue (should be equal to CASCI energy)
sectors = taper_qubits(qubit_H)
qubit_H, min_eigenvalue = sector_with_ground(sectors)
m = count_qubits(qubit_H)

print(min_eigenvalue, '\n')
print(qubit_H)

-1.2722647705358483 

12.052315964211402 [] +
-0.020087334934946778 [X0] +
0.0200873349349468 [X0 X1 X2] +
-0.014762808554334188 [X0 X1 Z2 X3] +
0.020087334934946778 [X0 X1 Z2 Z3] +
-0.014762808554334188 [X0 X1 X3] +
0.020087334934946778 [X0 X1 Z3] +
0.0200873349349468 [X0 Y1 Y2] +
0.014762808554334214 [X0 Z1 X2 X3] +
-0.0200873349349468 [X0 Z1 X2 Z3] +
0.02008733493494681 [X0 Z1 Z2] +
-0.014762808554334213 [X0 Z1 X3] +
0.02008733493494681 [X0 Z1 Z3] +
0.014762808554334214 [X0 X2 X3] +
-0.0200873349349468 [X0 X2 Z3] +
0.014762808554334171 [X0 Z2 X3] +
-0.020087334934946778 [X0 Z2 Z3] +
-0.014762808554334188 [Y0 X1 X2 Y3] +
-0.014762808554334188 [Y0 X1 Y2 X3] +
0.0200873349349468 [Y0 X1 Y2 Z3] +
0.014762808554334188 [Y0 Y1 X2 X3] +
-0.0200873349349468 [Y0 Y1 X2 Z3] +
-0.014762808554334188 [Y0 Y1 Y2 Y3] +
-0.014762808554334188 [Y0 Y1 Z2 X3] +
0.02008733493494681 [Y0 Y1 Z2 Z3] +
-0.014762808554334188 [Y0 Y1 X3] +
0.02008733493494681 [Y0 Y1 Z3] +
0.014762808554334171 [Y0 Z1 Y2 X3] +
-0.020

In [28]:
#set sampler to perform the annealing
# !pip install dwave-ocean-sdk
from neal import SimulatedAnnealingSampler
sampler = SimulatedAnnealingSampler() #uses simulated annealing, see D-Wave's ocean sdk for more options

In [29]:
### XBK method ###

#set r value
r = 4

#construct qubit Hamiltonians and C terms for XBK method
qubit_Hs, qubit_Cs = [],[]
for p in range(int(math.ceil(r/2+1))):
    qubit_Hs += [XBK_transform(qubit_H, r, p)]
    qubit_Cs += [construct_C(m, r, p)]

#run XBK method
XBK_energy, ground_state = XBK(qubit_Hs, qubit_Cs, r, sampler, starting_lam=0, num_samples=1000, strength=1e3, verbose=False)

print(XBK_energy)
print(ground_state) #ground state in rm-qubit space

-1.1852164199634
[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]


In [30]:
### QCC method ###

#set number of Bloch angle and entangler amplitude foldings
angle_folds = 2
amplitude_folds = 1

#create dictionary of QubitOperator entanglers
entanglers = {'IYZI': QubitOperator('Y1 Z2'), 'IZYI': QubitOperator('Z1 Y2'),
              'IXYI': QubitOperator('X1 Y2'), 'IYXI': QubitOperator('Y1 X2')}

#run QCC method
QCC_energy, variables = QCC(qubit_H, entanglers, angle_folds, amplitude_folds, sampler, num_cycles=10, num_samples=1000, strength=1e3)
    
print(QCC_energy)
print(variables)

NameError: name 'expr_to_dict' is not defined